In [2]:
import numpy as np 
from sets import Set
from conf_ope_rl import *
from __future__ import division 

# Note: we use Kallus & Zhou's Python 2 code so this notebook is Python 2

# Introduce my environments (ported from python 3...)

In [3]:
def translate_P(oldP):
    nStates = oldP.shape[2]
    nActions = oldP.shape[1]
    newP = np.zeros((nStates*2, nActions, nStates*2))
    for a in range(nActions):
        for x in range(nStates):
            newx = x*2
            newP[ newx , a , : ] = np.repeat( oldP[0, a, x, :], 2 )/2
            newP[ newx+1, a, : ] = np.repeat( oldP[1, a, x, :], 2 )/2
    return newP

def translate_Pi(oldPi):
    nStates = oldPi.shape[1]
    nActions = oldPi.shape[2]
    newPi = np.zeros((nActions, nStates*2))
    for x in range(nStates):
        newx = x*2
        newPi[:, newx] = oldPi[0, x, :]
        newPi[:, newx+1] = oldPi[1, x, :]
    return newPi

def translate_dist(oldDist):
    return np.repeat(oldDist, 2)/2

def translate_R(oldR):
    newR = np.zeros(oldR.shape)
    for t in range(200):
        newR += (0.95**t * oldR)
    return newR

### ope-graph

In [4]:
graphP = np.array([[[[0.  , 0.98, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.98, 0.02, 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.98, 0.02, 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.  , 0.  , 0.98, 0.02, 0.  ],
         [0.  , 0.  , 0.  , 0.  , 0.  , 0.98, 0.02, 0.  ],
         [0.25  , 0.25  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.5  ],
         [0.25  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75  ],
         [0.02  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.98  ]],

        [[0.  , 0.48, 0.52, 0.  , 0.  , 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.48, 0.52, 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.48, 0.52, 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.  , 0.  , 0.48, 0.52, 0.  ],
         [0.  , 0.  , 0.  , 0.  , 0.  , 0.48, 0.52, 0.  ],
         [0.25  , 0.25  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.5  ],
         [0.25  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75  ],
         [0.48  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.52  ]]],


       [[[0.  , 0.52, 0.48, 0.  , 0.  , 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.52, 0.48, 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.52, 0.48, 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.  , 0.  , 0.52, 0.48, 0.  ],
         [0.  , 0.  , 0.  , 0.  , 0.  , 0.52, 0.48, 0.  ],
         [0.25  , 0.25  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.5  ],
         [0.25  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75  ],
         [0.48  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.52  ]],

        [[0.  , 0.02, 0.98, 0.  , 0.  , 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.02, 0.98, 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.02, 0.98, 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.98, 0.  ],
         [0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.98, 0.  ],
         [0.25  , 0.25  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.5  ],
         [0.25  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75  ],
         [0.02  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.98  ]]]])
graphR = np.array([1., -1.,  1., -1.,  1., -1.,  1., -0.])

nStates = graphP.shape[2]
nActions = graphP.shape[1]

In [5]:
graphPi = np.array([[[0.86363636, 0.13636364],
        [0.86363636, 0.13636364],
        [0.86363636, 0.13636364],
        [0.86363636, 0.13636364],
        [0.86363636, 0.13636364],
        [0.7       , 0.3       ],
        [0.55882353, 0.44117647],
        [0.55882353, 0.44117647]],

       [[0.31818182, 0.68181818],
        [0.31818182, 0.68181818],
        [0.31818182, 0.68181818],
        [0.31818182, 0.68181818],
        [0.31818182, 0.68181818],
        [0.55882353, 0.44117647],
        [0.7       , 0.3       ],
        [0.7       , 0.3       ]]])

In [6]:
newGraphP = translate_P(graphP)

newGraphPi = translate_Pi(graphPi)

graphStateDist = np.array([1., 0., 0., 0., 0., 0., 0., 0.])

newGraphStateDist = translate_dist(graphStateDist)

gamma = 0.95

graphPhi = translate_R(graphR)

graphEval = np.array([[0.3, 0.7],
                   [0.3, 0.7],
                   [0.3, 0.7],
                   [0.3, 0.7],
                   [0.3, 0.7],
                   [0.3, 0.7],
                   [0.3, 0.7],
                   [0.3, 0.7]])

p_e_su = translate_Pi(np.array([graphEval,graphEval]))

In [8]:
tight = True; quiet = True; sense_min = False
nS = int(nStates*2)
nA = int(nActions)
nU = 2
nSmarg = int(nStates)

logGams_full = np.log([1.01, 1.5, 2, 10])
ngams = len(logGams_full)

# simulate
[ stateChangeHist, stateHist, a_s, s_a_sprime, p_infty_b_su_long, distr_hist ] =  simulate_rollouts( 
    nS, nA, newGraphP, newGraphPi, newGraphStateDist, 20000 )
p_infty_b_s = (reshape_byxrow(p_infty_b_su_long.T, nU).T).flatten()
[ p_a1_su, joint_s_a_sprime, s_a_giv_sprime ] = get_auxiliary_info_from_traj(stateChangeHist, 
                                        stateHist, a_s, s_a_sprime, p_infty_b_su_long, distr_hist, nA,nS)
# agg history and process
[aggStateHist, p_a1_s, p_e_s, agg_s_a_sprime, joint_s_a_sprime_agg, s_a_giv_sprime_agg] = agg_history(
            stateHist, s_a_sprime, p_infty_b_s, a_s, p_e_su, nA, nS, nSmarg, nU)

for ind,logGam in enumerate(logGams_full): 
    sense_min = True
    [a_bnd, b_bnd] = get_bnds_as( p_a1_s, logGam )
    [objVal, w_, m] = primal_opt_outer_L1_test_function(gamma, graphPhi, a_bnd,b_bnd, s_a_giv_sprime_agg, p_infty_b_s, p_e_s, p_a1_s, nSmarg, nA, tight, sense_min, quiet)

    print objVal

[0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
None
3.78241046098
2.52533465992
-6.60281596705


In [24]:
1/b_bnd.T

array([[0.06200873, 0.01508881, 0.01640399, 0.04618315],
       [0.05754796, 0.0140264 , 0.01935484, 0.04724615],
       [0.05872888, 0.01560624, 0.01492537, 0.05053599],
       [0.06721536, 0.01129944, 0.01129944, 0.05684755],
       [0.04376784, 0.01728785, 0.02408638, 0.04867076],
       [0.04850746, 0.01470588, 0.02516725, 0.04639175],
       [0.0509915 , 0.01485966, 0.02497027, 0.0440678 ],
       [0.04507711, 0.0156438 , 0.014555  , 0.06586022],
       [0.05024462, 0.01596233, 0.02367824, 0.04479628],
       [0.04272363, 0.01303739, 0.02651696, 0.05339806],
       [0.04503527, 0.01538462, 0.01463841, 0.06625578],
       [0.05103668, 0.0160517 , 0.01701323, 0.05379832],
       [0.04662905, 0.01502108, 0.02803249, 0.0440371 ],
       [0.04506474, 0.02634111, 0.01640259, 0.04564315],
       [0.04601617, 0.01566588, 0.01532668, 0.06314016],
       [0.06565091, 0.01521217, 0.01762402, 0.04136065]])

### check how long before steady state

In [11]:
def get_transitions(joint_s_a_sprime_agg, p_a1_s, p_b_infty_s,nS,nA): 
    P = np.zeros([nS,nA, nS])
    for s in range(P.shape[0]): 
        for a in range(nA): 
            P[s,a,:] = joint_s_a_sprime_agg[s,a,:] / (p_a1_s[a,s]*p_b_infty_s[s])
    return P
P_marg = get_transitions(joint_s_a_sprime_agg, p_a1_s, p_infty_b_s,nSmarg,nA)

P_x_to_xp = P_marg[:,0,:] * graphEval[0,0] + P_marg[:,1,:] * graphEval[0,1]
for x in range(nStates):
    P_x_to_xp[x] /= P_x_to_xp[x].sum()

def P_xt(horizon):
    result = graphStateDist
    for t in range(horizon):
        result = np.dot(result,P_x_to_xp)
    return result

np.dot(P_xt(200), graphPhi)

6.062907153675834

In [12]:
gamma = 0.95

def bellman(f):
    return graphR + gamma * (np.dot(P_x_to_xp,f))

test_f = np.zeros(nStates)
for _ in range(200):
    test_f = bellman(test_f)
print(test_f)

np.dot(test_f , graphStateDist)

[6.81567629 4.79490216 6.75856354 4.73724901 6.72426087 4.51062175
 6.74771787 5.78421601]


6.815676289567155

# Port over ope-gridworld from python 3

In [13]:
def gridworld_opetools(horizon = 100, slip = 0.05, confound_weight=0.1):
    tx,R,x_dist = infty_gridworld_ope_tools(horizon = horizon, slip = slip)

    V = rand_pi_val(tx, R, x_dist, 100)
    P = confound_V(tx, x_dist, V, confound_weight=confound_weight)
 
    u_dist = np.array([0.5, 0.5])
    gamma = 0.99

    nStates = tx.shape[1]
    nActions = tx.shape[0]

    pi = np.zeros((nStates, nActions))
    for s in range(nStates):
        pi[s] = [0.4, 0.1, 0.1, 0.4]
    pi_u = confound_pi_V(pi, tx, V, 0.2)
    return pi_u, P, R, x_dist, u_dist, gamma


In [14]:
def rand_pi_val(tx, R, x_dist, horizon):
    nActions = tx.shape[0]
    nStates = tx.shape[1]
    rand_pi = np.zeros((nStates, nActions))
    for i in range(nStates):
        rand_pi[i,:] = [1/nActions for a in range(nActions)]
    gamma = 0.98
    
    def bellman_eval_update(f, pi):
        Tf = np.zeros(f.shape)
        for s in range(nStates):
            for a in range(nActions):
                f_pi = np.array([np.dot(pi[xp] , f[xp, :]) for xp in range(nStates)])
                Tf[s,a] = np.dot(tx[a, s] , (R[a, s] + gamma * f_pi) )
        return Tf
    
    def bellman_eval(pi, horizon):
        Q = np.zeros((nStates, nActions))
        for k in range(horizon):
            Q = bellman_eval_update(Q, pi)
        return Q
    
    def get_value(Q, pi):
        V = np.array([np.dot(Q[x] , pi[x]) for x in range(nStates)])
        avgV = np.dot(V , x_dist)
        return V, avgV
    
    Q = bellman_eval(rand_pi, horizon)
    V, _ = get_value(Q, rand_pi)
    return V

In [15]:
def confound_V(tx, x_dist, V, confound_weight=0.1):
    nActions = tx.shape[0]
    nStates = tx.shape[1]
    
    P = np.zeros((2, nActions, nStates, nStates))
    for i in range(nStates):
        # which V is higher
        va = np.array([np.dot(tx[a, i] , V) for a in range(nActions)])

        minaction = va.argmin()
        maxaction = va.argmax()

        # find the highest value action transition
        # in u = 0 shift all actions towards the highest action probs
        # find the lowest value action transition
        # in u = 1 shift all actions towards the lowest value
        for a in range(nActions):
            P[0, a, i, :] = (1-confound_weight)*tx[a, i] + confound_weight*tx[maxaction, i]
            P[1, a, i, :] = (1-confound_weight)*tx[a, i] + confound_weight*tx[minaction, i]
    return P

In [16]:
def confound_pi_V(pi, tx, V, confound_weight):
    nStates = tx.shape[1]
    nActions = tx.shape[0]
    pi_u = np.zeros((2, nStates, nActions))
    for i in range(nStates):
        va = np.array([np.dot(tx[a, i] , V) for a in range(nActions)])
        maxaction = va.argmax() 
    
        for a in range(nActions):
            if a == maxaction:
                pi_u[0, i, a] = pi[i, a] + confound_weight
                pi_u[1, i, a] = pi[i, a] - confound_weight
            else:
                pi_u[0, i, a] = pi[i, a] - confound_weight
                pi_u[1, i, a] = pi[i ,a] + confound_weight
        
            if pi_u[0, i, a] < 0:
                pi_u[0, i, a] = 0
            if pi_u[1, i, a] < 0:
                pi_u[1, i, a] = 0
        
        # need to guarantee overlap:
        pi_u[0,i,:] += 0.05
        pi_u[1,i,:] += 0.05
        pi_u[0, i, :] /= pi_u[0, i, :].sum()
        pi_u[1, i, :] /= pi_u[1, i, :].sum()
    return pi_u


In [17]:
def infty_gridworld_ope_tools(horizon = 100, slip = 0.05):
    h = -0.5
    f = -0.005

    grid = np.array(
        [[-0.01, -0.01, -0.01, -0.01],
         [-0.01, -0.01, f    , h    ],
         [-0.01, h    , -0.01, h    ],
         [-0.01, h    , f    , +1   ]])
    gridlen = grid.shape[0]
    
    nStates = np.prod(grid.shape)
    nActions = 4

    tx = np.zeros((nActions, nStates, nStates))
    for i in range(nStates):
        if i % grid.shape[1] == 0:
            tx[0, i, i] += 1 - 3*slip
            tx[1, i, i] += slip
            tx[2, i, i] += slip
            tx[3, i, i] += slip
        else:
            tx[0, i, i-1] += 1 - 3*slip
            tx[1, i, i-1] += slip
            tx[2, i, i-1] += slip
            tx[3, i, i-1] += slip
        if i % grid.shape[1] == gridlen-1:
            tx[1, i, i] += 1 - 3*slip
            tx[0, i, i] += slip
            tx[2, i, i] += slip
            tx[3, i, i] += slip
        else:
            tx[1, i, i+1] += 1 - 3*slip
            tx[0, i, i+1] += slip
            tx[2, i, i+1] += slip
            tx[3, i, i+1] += slip
        if i // grid.shape[0] == 0:
            tx[2, i, i] += 1 - 3*slip
            tx[0, i, i] += slip
            tx[1, i, i] += slip
            tx[3, i, i] += slip
        else:
            tx[2, i, i-gridlen] += 1 - 3*slip
            tx[0, i, i-gridlen] += slip
            tx[1, i, i-gridlen] += slip
            tx[3, i, i-gridlen] += slip
        if i // grid.shape[0] == gridlen-1:
            tx[3, i, i] += 1 - 3*slip
            tx[0, i, i] += slip
            tx[1, i, i] += slip
            tx[2, i, i] += slip
        else:
            tx[3, i, i+gridlen] += 1 - 3*slip
            tx[0, i, i+gridlen] += slip
            tx[1, i, i+gridlen] += slip
            tx[2, i, i+gridlen] += slip
            
    R = np.zeros((nActions, nStates, nStates))
    for i in range(nStates):
        R[:, i, :] = grid.flatten()
    R[:, -1, -1] = 0

    x_dist = np.zeros(nStates)
    init_pos = [0, 1, 2, 3, 4, 8, 12]
    x_dist[init_pos] = 1/len(init_pos)
    
    return tx, R, x_dist

In [18]:
horizon = 8
pi_b, P, R, x_dist, u_dist, _ = gridworld_opetools(horizon = horizon, slip = 0.04, confound_weight=0.6)

gamma = 0.95

nStates = P.shape[2]
nActions = P.shape[1]

pi_e = np.zeros((nStates, nActions))
for i in range(nStates):
    pi_e[i] = [0.4, 0.1, 0.4, 0.1]

In [19]:
newP = translate_P(P)

newPi = translate_Pi(pi_b)

newStateDist = translate_dist(x_dist)

gamma = 0.95

phi = translate_R(R[0,0,:])

p_e_su = translate_Pi(np.array([pi_e,pi_e]))

In [21]:
tight = True; quiet = True; sense_min = False
nS = int(nStates*2)
nA = int(nActions)
nU = 2
nSmarg = int(nStates)

logGams_full = np.log([1.01])
ngams = len(logGams_full)

# simulate
[ stateChangeHist, stateHist, a_s, s_a_sprime, p_infty_b_su_long, distr_hist ] =  simulate_rollouts( 
    nS, nA, newP, newPi, newStateDist, 20000 )
p_infty_b_s = (reshape_byxrow(p_infty_b_su_long.T, nU).T).flatten()
[ p_a1_su, joint_s_a_sprime, s_a_giv_sprime ] = get_auxiliary_info_from_traj(stateChangeHist, 
                                        stateHist, a_s, s_a_sprime, p_infty_b_su_long, distr_hist, nA,nS)
# agg history and process
[aggStateHist, p_a1_s, p_e_s, agg_s_a_sprime, joint_s_a_sprime_agg, s_a_giv_sprime_agg] = agg_history(
            stateHist, s_a_sprime, p_infty_b_s, a_s, p_e_su, nA, nS, nSmarg, nU)

for ind,logGam in enumerate(logGams_full): 
    sense_min = True
    [a_bnd, b_bnd] = get_bnds_as( p_a1_s, logGam )
    [objVal, w_, m] = primal_opt_outer_L1_test_function(gamma, phi, a_bnd,b_bnd, s_a_giv_sprime_agg, p_infty_b_s, p_e_s, p_a1_s, nSmarg, nA, tight, sense_min, quiet)

    print objVal

[0.07142857 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857
 0.07142857 0.07142857 0.07142857 0.07142857 0.         0.
 0.         0.         0.         0.         0.07142857 0.07142857
 0.         0.         0.         0.         0.         0.
 0.07142857 0.07142857 0.         0.         0.         0.
 0.         0.        ]
-1.47463848698


In [22]:
def get_transitions(joint_s_a_sprime_agg, p_a1_s, p_b_infty_s,nS,nA): 
    P = np.zeros([nS,nA, nS])
    for s in range(P.shape[0]): 
        for a in range(nA): 
            P[s,a,:] = joint_s_a_sprime_agg[s,a,:] / (p_a1_s[a,s]*p_b_infty_s[s])
    return P
P_marg = get_transitions(joint_s_a_sprime_agg, p_a1_s, p_infty_b_s,nSmarg,nA)

P_x_to_xp = P_marg[:,0,:] * pi_e[0,0] + P_marg[:,1,:] * pi_e[0,1] + P_marg[:,2,:] * pi_e[0,2] + P_marg[:,3,:] * pi_e[0,3]
for x in range(nStates):
    P_x_to_xp[x] /= P_x_to_xp[x].sum()

def P_xt(horizon):
    result = x_dist
    for t in range(horizon):
        result = np.dot(result,P_x_to_xp)
    return result

np.dot(P_xt(200), phi)

-1.423655354607178

In [29]:
P_xt(200)

array([0.29951774, 0.07115334, 0.01947375, 0.01187345, 0.20049417,
       0.05976176, 0.01026022, 0.01889456, 0.1020118 , 0.03979711,
       0.00998739, 0.0077841 , 0.05288352, 0.06404138, 0.02753158,
       0.00453414])

In [30]:
gamma = 0.95

def bellman(f):
    return R[0,0,:] + gamma * (np.dot(P_x_to_xp,f))

test_f = np.zeros(nStates)
for _ in range(200):
    test_f = bellman(test_f)
print(test_f)

np.dot(test_f , x_dist)

[-0.9785691  -1.12181143 -1.38249451 -1.89792389 -1.06449245 -1.38784019
 -1.72793701 -2.86322955 -1.30744957 -2.16825624 -2.03881651 -2.73008264
 -1.7256562  -2.70714625 -2.2430098  -1.19543812]


-1.3540567360403393